In [ ]:
import tensorflow as tf
import os
import pickle
import numpy as np
import CifarData

CIFAR_DIR = "./../cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

In [ ]:
"""
testing
""" 

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData.CifarData(train_filenames, True)
test_data = CifarData.CifarData(test_filenames, False)

In [ ]:
def separable_conv_block(x,
                         output_channel_number,
                         name):
    """separable_conv block implementation"""
    """
    Args:
    - x:
    - output_channel_number: output channel of 1*1 conv layer.
    - name:
    """
    with tf.variable_scope(name):
        # get channel number:
        input_channel = x.get_shape().as_list()[-1]
        
        # split channels to a channel list:
        # channel_wise_x: [channel1, channel2, ...]
        channel_wise_x = tf.split(x, input_channel, axis = 3)
        
        # apply conv to each channel:
        output_channels = []
        for i in range(len(channel_wise_x)):
            output_channel = tf.layers.conv2d(channel_wise_x[i],
                                              1,
                                              (3, 3),
                                              strides = (1,1),
                                              padding = 'same',
                                              activation = tf.nn.relu,
                                              name = 'conv_%d' % i)
            output_channels.append(output_channel)
        
        # concat along channel(index=3)
        concat_layer = tf.concat(output_channels, axis = 3)
        
        # pass through a 1*1 conv layer
        conv1_1 = tf.layers.conv2d(concat_layer,
                                   output_channel_number,
                                   (1,1),
                                   strides = (1,1),
                                   padding = 'same',
                                   activation = tf.nn.relu,
                                   name = 'conv1_1')
    return conv1_1

In [ ]:
"""
construct the graph
"""
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# conv1: 神经元图， feature_map, 输出图像
conv1 = tf.layers.conv2d(x_image,
                         32, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv1')

pooling1 = tf.layers.max_pooling2d(conv1,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool1')

separable_2a = separable_conv_block(pooling1,
                                    32,
                                    name = 'separable_2a')
separable_2b = separable_conv_block(separable_2a,
                                    32,
                                    name = 'separable_2b')

pooling2 = tf.layers.max_pooling2d(separable_2b,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool2')

separable_3a = separable_conv_block(pooling2,
                                    32,
                                    name = 'separable_3a')
separable_3b = separable_conv_block(separable_3a,
                                    32,
                                    name = 'separable_3b')

pooling3 = tf.layers.max_pooling2d(separable_3b,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool3')


flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [ ]:
"""
run the graph
"""
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 100

# train 10k: 63.3%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
                                        [loss, accuracy, train_op],
                                        feed_dict={
                                            x: batch_data,
                                            y: batch_labels}
                                        )
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f'
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData.CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                                        [accuracy],
                                        feed_dict = {
                                            x: test_batch_data, 
                                            y: test_batch_labels
                                        })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f'% (i+1, test_acc))
                
                
            